In [ ]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # 이미지 위젯 초기화

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)


In [ ]:
import cv2
import imutils
import datetime

# avg 변수는 기준 프레임(배경)을 저장하여 새로운 프레임과 비교합니다.
avg = None

lastMovtionCaptured = datetime.datetime.now()

def motionDetect(imgInput):
    global avg, lastMovtionCaptured
    
    # 현재 타임스탬프를 가져옵니다.
    timestamp = datetime.datetime.now()
    
    # 프레임을 그레이스케일로 변환합니다.
    gray = cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY)
    
    # 가우시안 블러를 적용하여 노이즈를 줄입니다.
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # 기준 프레임(배경)이 없으면 현재 프레임을 기준 프레임으로 설정합니다.
    if avg is None:
        avg = gray.copy().astype("float")
        return imgInput

    # 배경 업데이트
    cv2.accumulateWeighted(gray, avg, 0.5)
    
    # 현재 프레임과 기준 프레임의 차이를 계산합니다.
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))

    # 변화가 있는 영역을 이진화합니다.
    thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # 변화가 있는 모든 영역을 찾고, 그 영역을 사각형으로 표시합니다.
    for c in cnts:
        if cv2.contourArea(c) < 30:
            continue

        (mov_x, mov_y, mov_w, mov_h) = cv2.boundingRect(c)
        cv2.rectangle(imgInput, (mov_x, mov_y), (mov_x + mov_w, mov_y + mov_h), (128, 255, 0), 1)

        lastMovtionCaptured = timestamp

    # 움직임을 감지한 후 0.5초 동안 움직임이 감지된 것으로 간주합니다.
    if (timestamp - lastMovtionCaptured).seconds >= 0.5:
        cv2.putText(imgInput, "Motion Detecting", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 255, 0), 1, cv2.LINE_AA)
    else:
        cv2.putText(imgInput, "Motion Detected", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 128, 255), 1, cv2.LINE_AA)
    
    return imgInput


In [ ]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(motionDetect(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')


In [ ]:
camera.unobserve(execute, names='value')
camera.stop()
